In [ ]:
import tsim
import numpy as np
from typing import Literal
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
theta = -np.arccos(np.sqrt(1 / 3)) / np.pi  # Distillation angle

c = tsim.Circuit(
    f"""
    R 0 
    R_X({theta}) 0
    T_DAG 0
    T 0
    R_X({-theta}) 0
    M 0
    """
)
sampler = c.compile_sampler()
samples = sampler.sample(10_000_000, batch_size=1_000_000)
print(f"Fidelity: {1 - np.count_nonzero(samples) / len(samples)}")
c.diagram("timeline-svg", height=100)

In [ ]:
p = 0.05
c = tsim.Circuit(
    f"""
    R 0 
    R_X({theta}) 0
    T_DAG 0
    DEPOLARIZE1({p}) 0
    T 0
    R_X({-theta}) 0
    M 0
    """
)
sampler = c.compile_sampler()
samples = sampler.sample(100_000, batch_size=100_000)
print(f"Fidelity: {1 - np.count_nonzero(samples) / len(samples)}")
c.diagram("timeline-svg", height=100)

In [ ]:
def make_logical_distillation_circuit(angle: float):
    return tsim.Circuit(
        f"""
        # initial state
        R 0 1 2 3 4
        R_X({angle}) 0 1 2 3 4
        T_DAG 0 1 2 3 4
        DEPOLARIZE1({p}) 0 1 2 3 4


        # distillation
        SQRT_X 0 1 4
        CZ 0 1 2 3
        SQRT_Y 0 3
        CZ 0 2 3 4
        TICK
        SQRT_X_DAG 0
        CZ 0 4 1 3
        TICK
        SQRT_X_DAG 0 1 2 3 4

        # undo
        T 0
        R_X({-angle}) 0

        M 0 1 2 3 4
        """
    )


c = make_logical_distillation_circuit(theta)
c.diagram("timeline-svg", height=240)

In [ ]:
sampler = c.compile_sampler()
sampler

In [ ]:
samples = sampler.sample(10_000, batch_size=10_000)


def post_select(samples: np.ndarray):
    distilled_output = samples[:, 0]
    distillation_syndromes = samples[:, 1:]
    sel = np.all(distillation_syndromes == np.array([1, 0, 1, 1]), axis=1)
    return distilled_output[sel]


post_selected_samples = post_select(samples)
print(
    f"Fidelity: {1 - np.count_nonzero(post_selected_samples) / len(post_selected_samples):.5f}"
)
print(
    f"Percentage of post-selected samples: {len(post_selected_samples) / len(samples) * 100:.2f}%"
)

In [ ]:
angles = np.linspace(-1, 1, 25) + theta
fidelities = []
post_selection_rate = []
for a in tqdm(angles):
    c = make_logical_distillation_circuit(a)
    sampler = c.compile_sampler()
    samples = sampler.sample(5_000, batch_size=5_000)

    post_selected_samples = post_select(samples)
    post_selection_rate.append(len(post_selected_samples) / len(samples))
    f = np.count_nonzero(post_selected_samples) / len(post_selected_samples)
    fidelities.append(1 - f)

In [ ]:
fig, ax1 = plt.subplots()

(line1,) = ax1.semilogy(
    angles - theta, fidelities, "*-", c="g", lw=0.5, label="Fidelity"
)
ax1.axvline(0, linestyle="--", lw=0.5)
ax1.set_xlabel(r"$(\theta - \theta^*)/\pi$")
ax1.set_ylabel("Fidelity")
ax1.set_ylim(np.min(fidelities) - 0.1, 1)
ax1.set_xlim(-1, 1)

ax2 = ax1.twinx()
(line2,) = ax2.plot(
    angles - theta,
    post_selection_rate,
    ".-",
    c="orange",
    lw=0.5,
    label="Post-selection rate",
)
ax2.set_ylabel("Post-selection rate")
ax2.legend(handles=[line1, line2], loc="lower center");

In [ ]:
from utils.codes import ColorEncoder5


def make_circuit(basis: Literal["X", "Y", "Z"]):
    encoder = ColorEncoder5()
    encoder.initialize(
        f"""
        R 0 1 2 3 4
        R_X({theta}) 0 1 2 3 4
        T_DAG 0 1 2 3 4
        DEPOLARIZE1({p}) 0 1 2 3 4
        """
    )
    encoder.append(
        """
        SQRT_X 0 1 4
        CZ 0 1 2 3
        SQRT_Y 0 3
        CZ 0 2 3 4
        TICK
        SQRT_X_DAG 0
        CZ 0 4
        TICK
        CZ 1 3
        TICK
        SQRT_X_DAG 0 1 2 3 4
        DEPOLARIZE1(0.0) 0 1 2 3 4
        """
        + ("H 0" if basis == "X" else "H_YZ 0" if basis == "Y" else "")
        + """
        M 0 1 2 3 4
        DETECTOR rec[-5]
        DETECTOR rec[-4]
        DETECTOR rec[-3]
        DETECTOR rec[-2]
        DETECTOR rec[-1]
        OBSERVABLE_INCLUDE(0) rec[-5]
        OBSERVABLE_INCLUDE(1) rec[-4]
        OBSERVABLE_INCLUDE(2) rec[-3]
        OBSERVABLE_INCLUDE(3) rec[-2]
        OBSERVABLE_INCLUDE(4) rec[-1]
        """
    )
    return encoder.circuit


c = make_circuit("Z")
print(
    f"""Qubits: {c.num_qubits}
Detectors: {c.num_detectors}
Observables: {c.num_observables}"""
)
c.diagram("timeline-svg", height=500)

In [ ]:
circ_x, circ_y, circ_z = make_circuit("X"), make_circuit("Y"), make_circuit("Z")
sampler_x = c.compile_detector_sampler()
sampler_y = c.compile_detector_sampler()
sampler_z = c.compile_detector_sampler()

In [ ]:
def sample_and_post_select(sampler, shots=100_000):
    det_samples, obs_samples = sampler.sample(
        shots=shots, batch_size=100_000, separate_observables=True
    )
    assert np.count_nonzero(det_samples) == 0

    post_selected_observables = post_select(obs_samples)
    p1 = np.count_nonzero(post_selected_observables) / len(post_selected_observables)
    eval = 1 - 2 * p1
    print(
        f"Expectation value: {eval:.4f}, percentage kept: {len(post_selected_observables) / len(obs_samples) * 100:.2f}%"
    )
    return 1 - 2 * p1

In [ ]:
vx = sample_and_post_select(sampler_x)
vy = sample_and_post_select(sampler_y)
vz = sample_and_post_select(sampler_z)

In [ ]:
id = np.eye(2)
x = np.array([[0, 1], [1, 0]])
y = np.array([[0, -1j], [1j, 0]])
z = np.array([[1, 0], [0, -1]])

rho = (id + vx * x + vy * y + vz * z) / 2

In [ ]:
a = theta * np.pi
rx = np.array(
    [[np.cos(a / 2), -1j * np.sin(a / 2)], [-1j * np.sin(a / 2), np.cos(a / 2)]]
)
t_dag = np.array([[1, 0], [0, np.exp(-1j * np.pi / 4)]])
psi_expected = t_dag @ rx @ np.array([1, 0])

In [ ]:
fidelity = psi_expected.conj().T @ rho @ psi_expected
print(f"Fidelity: {fidelity.real:.5f}")